# Notebook showcasing the NIFTy 6 Correlated Field model

**Skip to `Parameter Showcases` for the meat/veggies ;)**

The field model roughly works like this:

`f = HT( A * zero_mode * xi ) + offset`

`A` is a spectral power field which is constructed from power spectra that hold on subdomains of the target domain.
It is scaled by a zero mode operator and then pointwise multiplied by a gaussian excitation field, yielding
a representation of the field in harmonic space.
It is then transformed into the target real space and a offset added.

The power spectra `A` is constructed of are in turn constructed as the sum of a power law component
and an integrated Wiener process whose amplitude and roughness can be set.

## Setup code

In [ ]:
import nifty7 as ift
import matplotlib.pyplot as plt
import numpy as np
ift.random.push_sseq_from_seed(43)

n_pix = 256
x_space = ift.RGSpace(n_pix)

In [ ]:
# Plotting routine
def plot(fields, spectra, title=None):
    # Plotting preparation is normally handled by nifty7.Plot
    # It is done manually here to be able to tweak details
    # Fields are assumed to have identical domains
    fig = plt.figure(tight_layout=True, figsize=(12, 3.5))
    if title is not None:
        fig.suptitle(title, fontsize=14)
    
    # Field
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.axhline(y=0., color='k', linestyle='--', alpha=0.25)
    for field in fields:
        dom = field.domain[0]
        xcoord = np.arange(dom.shape[0]) * dom.distances[0]
        ax1.plot(xcoord, field.val)
    ax1.set_xlim(xcoord[0], xcoord[-1])
    ax1.set_ylim(-5., 5.)
    ax1.set_xlabel('x')
    ax1.set_ylabel('f(x)')
    ax1.set_title('Field realizations')
    
    # Spectrum
    ax2 = fig.add_subplot(1, 2, 2)
    for spectrum in spectra:
        xcoord = spectrum.domain[0].k_lengths
        ycoord = spectrum.val_rw()
        ycoord[0] = ycoord[1]
        ax2.plot(xcoord, ycoord)
    ax2.set_ylim(1e-6, 10.)
    ax2.set_xscale('log')
    ax2.set_yscale('log')
    ax2.set_xlabel('k')
    ax2.set_ylabel('p(k)')
    ax2.set_title('Power Spectrum')
    
    fig.align_labels()
    plt.show()

# Helper: draw main sample
main_sample = None
def init_model(m_pars, fl_pars):
    global main_sample
    cf = ift.CorrelatedFieldMaker.make(**m_pars)
    cf.add_fluctuations(**fl_pars)
    field = cf.finalize(prior_info=0)
    main_sample = ift.from_random(field.domain)
    print("model domain keys:", field.domain.keys())
    
# Helper: field and spectrum from parameter dictionaries + plotting
def eval_model(m_pars, fl_pars, title=None, samples=None):
    cf = ift.CorrelatedFieldMaker.make(**m_pars)
    cf.add_fluctuations(**fl_pars)
    field = cf.finalize(prior_info=0)
    spectrum = cf.amplitude
    if samples is None:
        samples = [main_sample]
    field_realizations = [field(s) for s in samples]
    spectrum_realizations = [spectrum.force(s) for s in samples]
    plot(field_realizations, spectrum_realizations, title)

def gen_samples(key_to_vary):
    if key_to_vary is None:
        return [main_sample]
    dct = main_sample.to_dict()
    subdom_to_vary = dct.pop(key_to_vary).domain
    samples = []
    for i in range(8):
        d = dct.copy()
        d[key_to_vary] = ift.from_random(subdom_to_vary)
        samples.append(ift.MultiField.from_dict(d))
    return samples
        
def vary_parameter(parameter_key, values, samples_vary_in=None):
    s = gen_samples(samples_vary_in)
    for v in values:
        if parameter_key in cf_make_pars.keys():
            m_pars = {**cf_make_pars, parameter_key: v}
            eval_model(m_pars, cf_x_fluct_pars, f"{parameter_key} = {v}", s)
        else:
            fl_pars = {**cf_x_fluct_pars, parameter_key: v}
            eval_model(cf_make_pars, fl_pars, f"{parameter_key} = {v}", s)

## Before the Action: The Moment-Matched Log-Normal Distribution
Many properties of the correlated field are modelled as being lognormally distributed.

The distribution models are parametrized via their means `_mean` and standard-deviations `_stddev`.

To get a feeling of how the ratio of the `mean` and `stddev` parameters influences the distribution shape,
here are a few example histograms: (observe the x-axis!)

In [ ]:
fig = plt.figure(figsize=(13, 3.5))
mean = 1.0
sigmas = [1.0, 0.5, 0.1]

for i in range(3):
    op = ift.LognormalTransform(mean=mean, sigma=sigmas[i],
                                key='foo', N_copies=0)
    op_samples = np.array(
        [op(s).val for s in [ift.from_random(op.domain) for i in range(10000)]])

    ax = fig.add_subplot(1, 3, i + 1)
    ax.hist(op_samples, bins=50)
    ax.set_title(f"mean = {mean}, sigma = {sigmas[i]}")
    ax.set_xlabel('x')
    del op_samples

plt.show()

## The Neutral Field

To demonstrate the effect of all parameters, first a 'neutral' set of parameters
is defined which then are varied one by one, showing the effect of the variation
on the generated field realizations and the underlying power spectrum from which
they were drawn.

As a neutral field, a model with a white power spectrum and vanishing spectral power was chosen.

In [ ]:
# Neutral model parameters yielding a quasi-constant field
cf_make_pars = {
    'offset_mean': 0.,
    'offset_std_mean': 1e-3,
    'offset_std_std': 1e-16,
    'prefix': ''
}

cf_x_fluct_pars = {
    'target_subdomain': x_space,
    'fluctuations_mean': 1e-3,
    'fluctuations_stddev': 1e-16,
    'flexibility_mean': 1e-3,
    'flexibility_stddev': 1e-16,
    'asperity_mean': 1e-3,
    'asperity_stddev': 1e-16,
    'loglogavgslope_mean': 0.,
    'loglogavgslope_stddev': 1e-16
}

init_model(cf_make_pars, cf_x_fluct_pars)

In [ ]:
# Show neutral field
eval_model(cf_make_pars, cf_x_fluct_pars, "Neutral Field")

# Parameter Showcases

## The `fluctuations_` parameters of `add_fluctuations()`

determine the **amplitude of variations along the field dimension**
for which `add_fluctuations` is called.

`fluctuations_mean` set the _average_ amplitude of the fields fluctuations along the given dimension,\
`fluctuations_stddev` sets the width and shape of the amplitude distribution.

The amplitude is modelled as being log-normally distributed,
see `The Moment-Matched Log-Normal Distribution` above for details.

#### `fluctuations_mean`:

In [ ]:
vary_parameter('fluctuations_mean', [0.05, 0.5, 2.], samples_vary_in='xi')

#### `fluctuations_stddev`:

In [ ]:
cf_x_fluct_pars['fluctuations_mean'] = 1.0
vary_parameter('fluctuations_stddev', [0.01, 0.1, 1.0], samples_vary_in='fluctuations')

## The `loglogavgslope_` parameters of `add_fluctuations()`

determine **the slope of the loglog-linear (power law) component of the power spectrum**.

The slope is modelled to be normally distributed.

#### `loglogavgslope_mean`:

In [ ]:
vary_parameter('loglogavgslope_mean', [-3., -1., 1.], samples_vary_in='xi')

#### `loglogavgslope_stddev`:

In [ ]:
cf_x_fluct_pars['loglogavgslope_mean'] = -1.0
vary_parameter('loglogavgslope_stddev', [0.01, 0.1, 1.0], samples_vary_in='loglogavgslope')

## The `flexibility_` parameters of `add_fluctuations()`

determine **the amplitude of the integrated Wiener process component of the power spectrum**
(how strong the power spectrum varies besides the power-law).

`flexibility_mean` sets the _average_ amplitude of the i.g.p. component,
`flexibility_stddev` sets how much the amplitude can vary.\
These two parameters feed into a moment-matched log-normal distribution model,
see above for a demo of its behavior.

#### `flexibility_mean`:

In [ ]:
vary_parameter('flexibility_mean', [0.1, 1.0, 3.0], samples_vary_in='spectrum')

#### `flexibility_stddev`:

In [ ]:
cf_x_fluct_pars['flexibility_mean'] = 2.0
vary_parameter('flexibility_stddev', [0.01, 0.1, 1.0], samples_vary_in='flexibility')

## The `asperity_` parameters of `add_fluctuations()`

`asperity_` determines **how rough the integrated Wiener process component of the power spectrum is**.

`asperity_mean` sets the average roughness, `asperity_stddev` sets how much the roughness can vary.\
These two parameters feed into a moment-matched log-normal distribution model,
see above for a demo of its behavior.

#### `asperity_mean`:

In [ ]:
vary_parameter('asperity_mean', [0.001, 1.0, 5.0], samples_vary_in='spectrum')

#### `asperity_stddev`:

In [ ]:
cf_x_fluct_pars['asperity_mean'] = 1.0
vary_parameter('asperity_stddev', [0.01, 0.1, 1.0], samples_vary_in='asperity')

## The `offset_mean` parameter of `CorrelatedFieldMaker.make()`

The `offset_mean` parameter defines a global additive offset on the field realizations.

If the field is used for a lognormal model `f = field.exp()`, this acts as a global signal magnitude offset.

In [ ]:
# Reset model to neutral
cf_x_fluct_pars['fluctuations_mean'] = 1e-3
cf_x_fluct_pars['flexibility_mean'] = 1e-3
cf_x_fluct_pars['asperity_mean'] = 1e-3
cf_x_fluct_pars['loglogavgslope_mean'] = 1e-3

In [ ]:
vary_parameter('offset_mean', [3., 0., -2.])

## The `offset_std_` parameters of `CorrelatedFieldMaker.make()`

Variation of the global offset of the field are modelled as being log-normally distributed.
See `The Moment-Matched Log-Normal Distribution` above for details.

The `offset_std_mean` parameter sets how much NIFTy will vary the offset *on average*.\
The `offset_std_std` parameters defines the with and shape of the offset variation distribution.

#### `offset_std_mean`:

In [ ]:
vary_parameter('offset_std_mean', [1e-16, 0.5, 2.], samples_vary_in='xi')

#### `offset_std_std`:

In [ ]:
cf_make_pars['offset_std_mean'] = 1.0
vary_parameter('offset_std_std', [0.01, 0.1, 1.0], samples_vary_in='zeromode')